# CNN을 이용한 글자 인식 (MNIST Dataset)

In [1]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
import numpy as np

# CNN의 재료 준비

In [14]:
# 훈련 파라미터 설정
EPOCH = 10
batch_size = 512

# 이미지 크기 설정
img_rows, img_cols = 28, 28

# (훈련 이미지, 훈련 레이블), (테스트 이미지, 테스트 레이블)로 데이터 구분
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
print(f"x_train shape: {x_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test.shape}")
print(f"y_test shape: {y_test.shape}")
print("\n")

# data를 로드한 다음 28x28크기의 matrix를 input shape로 정의
# 28x28x1로 데이터의 reshape
input_shape = (img_rows, img_cols, 1)
print(f"input_shape: {input_shape}") # 28x28x1
x_train = x_train.reshape(x_train.shape[0], img_rows,img_cols,1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols,1)
print(f"x_train shape: {x_train.shape}")
print(f"x_test shape: {x_test.shape}")
print("\n")

# 훈련 이미지와 테스트 이미지 픽셀 값의 범위를 (0,1)로 변경
x_train = x_train.astype('float32')/255 # 정규화
x_test = x_test.astype('float32')/255 # 정규화
print(f"x_train shape(정규화): {x_train.shape}")
print(f"x_test shape(정규화): {x_test.shape}")

# train, test 데이터 갯수 및 shape
print("[# train, test 데이터 갯수 및 shape]")
print(x_train.shape[0], 'train samples') #28x28x1이 총 60,000개
print(x_test.shape[0], 'test samples') #28x28x1이 총 10,000개

x_train shape: (60000, 28, 28)
y_train shape: (60000,)
x_test shape: (10000, 28, 28)
y_test shape: (10000,)


input_shape: (28, 28, 1)
x_train shape: (60000, 28, 28, 1)
x_test shape: (10000, 28, 28, 1)


x_train shape(정규화): (60000, 28, 28, 1)
x_test shape(정규화): (10000, 28, 28, 1)
[# train, test 데이터 갯수 및 shape]
60000 train samples
10000 test samples


## CNN 모델의 구조 설정

In [15]:
# 분류할 class 개수
num_classes = 10

 # 레이블을 one-hot encoding 형식으로 변환
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(f"y_train : {y_train}")
print(f"y_test : {y_test}")

# CNN 모델 구조 설정
model = Sequential()

# Conv.
model.add(Conv2D(32, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu', input_shape=input_shape)) # out dem. ~> 28x28x32

# MaxPooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2))) # out dem. ~> 14x14x32

# Conv.
model.add(Conv2D(64, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')) # out dem. ~> 14x14x64

# MaxPooling
model.add(MaxPooling2D(pool_size=(2,2))) # out dem. ~> 7x7x64x60000

# Dropout for reduce overfitting
model.add(Dropout(0.25)) # 🤔 

# Flatten for use input layer of fully connected layer
model.add(Flatten()) # 7x7x64개 input nodes of fully connected layer

# like Hidden layer
model.add(Dense(1000, activation='relu'))

# Dropout for reduce overfitting
model.add(Dropout(0.5))

# like weighted sum of last hidden layer
model.add(Dense(num_classes, activation='softmax'))

#모델 구조 출력
print(f"model : {model.summary()}")

y_train : [[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]]
y_test : [[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 28, 28, 32)        320       
                                                                 
 max_pooling2d_10 (MaxPooli  (None, 14, 14, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_11 (Conv2D)          (None, 14, 14, 64)        18496     
                                                                 
 max_pooling2d_11 (MaxPooli  (None, 7, 7, 64)        

## 모델 훈련 및 테스트

In [16]:
# Loss function, Optimizer, 검증 척도(metric) 설정
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
 # 모델 훈련
hist = model.fit(x_train, y_train, batch_size=batch_size, epochs=EPOCH, validation_data=(x_test, y_test))
 # 학습 후 Test loss, accuracy 출력
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/10
  5/118 [>.............................] - ETA: 52s - loss: 1.8829 - accuracy: 0.5145

KeyboardInterrupt: 